# Classification de documents d'opinions

Membres du groupe E :

Darnala Baptiste

Di Giovanni Thomas (21505926)

Duverger Eliott

Pierre (dsl je connais pas ton nom de famille)

Importer les données :

In [ ]:
import pandas

data = pandas.read_csv('Data/dataset.csv', sep='\t', header = None, encoding = "utf8")


Pré-traitements à tester :

Suppressions des stopwords

Passage en minuscule

Supression de la ponctuation

Lemmatisation ou stemmatisation

Remplacement des nombres

Supression de caractères non ASCII

Suppression des contractions

N-grams (pour prendre en compte la négation)

Classifieurs :
